# 1. Imports

In [7]:
import importlib

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

In [3]:
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score

In [4]:
os.environ['TORCH'] = torch.__version__
print('torch version:', torch.__version__)

!pip3 install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip3 install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip3 install -q git+https://github.com/pyg-team/pytorch_geometric.git

torch version: 2.0.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 33.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, to_hetero
from torch_geometric.data import Data, HeteroData
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch_geometric.transforms as T

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cpu


# Main

In [9]:
import os
os.chdir("/content/drive/Shareddrives/NewsRec/")

In [12]:
!python3 main.py --data "cit_pt" --plm "ptbert" --use_seperate_test_data

Namespace(data='cit_pt', plm='ptbert', use_seperate_test_data=True, num_user_features=32, num_item_features=768, user_input_layer='lin', item_input_layer='lin', input_hidden_channels=64, encoder='SAGE', predictor='DOT', encoder_layers=2, predictor_layers=0, encoder_hidden_channels=64, predictor_hidden_channels=32, dropout=0.0, batch_size=1024, lr=0.001, epochs=-1, eval_steps=1, device=0, runs=1)
Device: cpu
--
users shape: (14417, 4)
unique (users, articles): (1619, 821)
contentId                                   310515487419366995
personId                                  -1130272294246983140
url          http://blog.runrun.it/erros-de-portugues-em-e-...
lang                                                        pt
Name: 0, dtype: object
--
items shape: (821, 4)
unique articles: 821
contentId                                 -9157338616628196758
title        Situação financeira ruim de varejistas pressio...
url          http://economia.ig.com.br/2016-03-27/situacao-...
lang          

In [13]:
!python3 main.py --data "cit_pt" --plm "mbert" --use_seperate_test_data

Namespace(data='cit_pt', plm='mbert', use_seperate_test_data=True, num_user_features=32, num_item_features=768, user_input_layer='lin', item_input_layer='lin', input_hidden_channels=64, encoder='SAGE', predictor='DOT', encoder_layers=2, predictor_layers=0, encoder_hidden_channels=64, predictor_hidden_channels=32, dropout=0.0, batch_size=1024, lr=0.001, epochs=-1, eval_steps=1, device=0, runs=1)
Device: cpu
--
users shape: (14417, 4)
unique (users, articles): (1619, 821)
contentId                                   310515487419366995
personId                                  -1130272294246983140
url          http://blog.runrun.it/erros-de-portugues-em-e-...
lang                                                        pt
Name: 0, dtype: object
--
items shape: (821, 4)
unique articles: 821
contentId                                 -9157338616628196758
title        Situação financeira ruim de varejistas pressio...
url          http://economia.ig.com.br/2016-03-27/situacao-...
lang           

In [14]:
!python3 main.py --data "cit_pt" --plm "xlm" --use_seperate_test_data

Namespace(data='cit_pt', plm='xlm', use_seperate_test_data=True, num_user_features=32, num_item_features=768, user_input_layer='lin', item_input_layer='lin', input_hidden_channels=64, encoder='SAGE', predictor='DOT', encoder_layers=2, predictor_layers=0, encoder_hidden_channels=64, predictor_hidden_channels=32, dropout=0.0, batch_size=1024, lr=0.001, epochs=-1, eval_steps=1, device=0, runs=1)
Device: cpu
--
users shape: (14417, 4)
unique (users, articles): (1619, 821)
contentId                                   310515487419366995
personId                                  -1130272294246983140
url          http://blog.runrun.it/erros-de-portugues-em-e-...
lang                                                        pt
Name: 0, dtype: object
--
items shape: (821, 4)
unique articles: 821
contentId                                 -9157338616628196758
title        Situação financeira ruim de varejistas pressio...
url          http://economia.ig.com.br/2016-03-27/situacao-...
lang             

In [16]:
!python3 main.py --data "cit_tr" --plm "enbert" --use_seperate_test_data

Namespace(data='cit_tr', plm='enbert', use_seperate_test_data=True, num_user_features=32, num_item_features=768, user_input_layer='lin', item_input_layer='lin', input_hidden_channels=64, encoder='SAGE', predictor='DOT', encoder_layers=2, predictor_layers=0, encoder_hidden_channels=64, predictor_hidden_channels=32, dropout=0.0, batch_size=1024, lr=0.001, epochs=-1, eval_steps=1, device=0, runs=1)
Device: cpu
--
users shape: (14417, 4)
unique (users, articles): (1619, 821)
contentId                                   310515487419366995
personId                                  -1130272294246983140
url          http://blog.runrun.it/erros-de-portugues-em-e-...
lang                                                        pt
Name: 0, dtype: object
--
items shape: (821, 3)
unique articles: 821
contentId                               -9160000000000000000.0
title        Bad financial situation of retailers puts pres...
url          http://economia.ig.com.br/2016-03-27/situacao-...
Name: 0, dtype

In [17]:
!python3 main.py --data "cit_en" --plm "enbert" --use_seperate_test_data

Namespace(data='cit_en', plm='enbert', use_seperate_test_data=True, num_user_features=32, num_item_features=768, user_input_layer='lin', item_input_layer='lin', input_hidden_channels=64, encoder='SAGE', predictor='DOT', encoder_layers=2, predictor_layers=0, encoder_hidden_channels=64, predictor_hidden_channels=32, dropout=0.0, batch_size=1024, lr=0.001, epochs=-1, eval_steps=1, device=0, runs=1)
Device: cpu
--
users shape: (26206, 4)
unique (users, articles): (1644, 2154)
contentId                      -3499919498720038879
personId                       -8845298781299428018
url          http://techcrunch.com/2016/06/07/hiri/
lang                                             en
Name: 0, dtype: object
--
items shape: (2154, 4)
unique articles: 2154
contentId                                 -6451309518266745024
title        Ethereum, a Virtual Currency, Enables Transact...
url          http://www.nytimes.com/2016/03/28/business/dea...
lang                                                   

In [21]:
!python3 main.py --data "cit_pten" --plm "mbert" --use_seperate_test_data

Namespace(data='cit_pten', plm='mbert', use_seperate_test_data=True, num_user_features=32, num_item_features=768, user_input_layer='lin', item_input_layer='lin', input_hidden_channels=64, encoder='SAGE', predictor='DOT', encoder_layers=2, predictor_layers=0, encoder_hidden_channels=64, predictor_hidden_channels=32, dropout=0.0, batch_size=1024, lr=0.001, epochs=-1, eval_steps=1, device=0, runs=1)
Device: cpu
--
users shape: (40623, 4)
unique (users, articles): (1894, 2975)
contentId                      -3499919498720038879
personId                       -8845298781299428018
url          http://techcrunch.com/2016/06/07/hiri/
lang                                             en
Name: 0, dtype: object
--
items shape: (2975, 4)
unique articles: 2975
contentId                                 -6451309518266745024
title        Ethereum, a Virtual Currency, Enables Transact...
url          http://www.nytimes.com/2016/03/28/business/dea...
lang                                                  

In [22]:
!python3 main.py --data "cit_pten" --plm "xlm" --use_seperate_test_data

Namespace(data='cit_pten', plm='xlm', use_seperate_test_data=True, num_user_features=32, num_item_features=768, user_input_layer='lin', item_input_layer='lin', input_hidden_channels=64, encoder='SAGE', predictor='DOT', encoder_layers=2, predictor_layers=0, encoder_hidden_channels=64, predictor_hidden_channels=32, dropout=0.0, batch_size=1024, lr=0.001, epochs=-1, eval_steps=1, device=0, runs=1)
Device: cpu
--
users shape: (40623, 4)
unique (users, articles): (1894, 2975)
contentId                      -3499919498720038879
personId                       -8845298781299428018
url          http://techcrunch.com/2016/06/07/hiri/
lang                                             en
Name: 0, dtype: object
--
items shape: (2975, 4)
unique articles: 2975
contentId                                 -6451309518266745024
title        Ethereum, a Virtual Currency, Enables Transact...
url          http://www.nytimes.com/2016/03/28/business/dea...
lang                                                    

In [20]:
!python3 main.py --data "cit_tren" --plm "enbert" --use_seperate_test_data

Namespace(data='cit_tren', plm='enbert', use_seperate_test_data=True, num_user_features=32, num_item_features=768, user_input_layer='lin', item_input_layer='lin', input_hidden_channels=64, encoder='SAGE', predictor='DOT', encoder_layers=2, predictor_layers=0, encoder_hidden_channels=64, predictor_hidden_channels=32, dropout=0.0, batch_size=1024, lr=0.001, epochs=-1, eval_steps=1, device=0, runs=1)
Device: cpu
--
users shape: (2956, 4)
unique (users, articles): (653, 2956)
contentId                      -3499919498720038879
personId                       -8845298781299428018
url          http://techcrunch.com/2016/06/07/hiri/
lang                                             en
Name: 0, dtype: object
--
items shape: (2956, 4)
unique articles: 2956
contentId                               -6451309518266744832.0
title        Ethereum, a Virtual Currency, Enables Transact...
url          http://www.nytimes.com/2016/03/28/business/dea...
lang                                                   

In [25]:
!python3 main.py --data "adressa_no" --plm "xlm" --use_seperate_test_data

Namespace(data='adressa_no', plm='xlm', use_seperate_test_data=True, num_user_features=32, num_item_features=768, user_input_layer='lin', item_input_layer='lin', input_hidden_channels=64, encoder='SAGE', predictor='DOT', encoder_layers=2, predictor_layers=0, encoder_hidden_channels=64, predictor_hidden_channels=32, dropout=0.0, batch_size=1024, lr=0.001, epochs=-1, eval_steps=1, device=0, runs=1)
Device: cpu
--
users shape: (1857389, 2)
unique (users, articles): (535259, 15721)
userId    cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov
id                 01c74ff7ec02862bed5f861cba7e7226dfd31beb
Name: 0, dtype: object
--
items shape: (15721, 2)
unique articles: 15721
id       6a0612e60690288a776834811004ce133f326cee
title         - Historiene er nesten for utrolige
Name: 0, dtype: object
Doing seperate train and test data for adressa_no
--
users shape: (455761, 2)
unique (users, articles): (237605, 6793)
userId           cx:ii6jgkx3p8662ju0:26gkbv29my998
id        f9ad33b317728440d5ed1

In [30]:
!python3 main.py --data "adressa_no" --plm "xlm" --use_seperate_test_data

Namespace(data='adressa_no', plm='xlm', use_seperate_test_data=True, num_user_features=32, num_item_features=768, user_input_layer='lin', item_input_layer='lin', input_hidden_channels=64, encoder='SAGE', predictor='DOT', encoder_layers=2, predictor_layers=0, encoder_hidden_channels=64, predictor_hidden_channels=32, dropout=0.0, batch_size=1024, lr=0.001, epochs=-1, eval_steps=1, device=0, runs=1)
Device: cpu
--
users shape: (1857389, 2)
unique (users, articles): (535259, 15721)
userId    cx:0049d95b7c3a854f8c1edb0a13197e71:1zqfs7vo2seov
id                 01c74ff7ec02862bed5f861cba7e7226dfd31beb
Name: 0, dtype: object
--
items shape: (15721, 2)
unique articles: 15721
id       6a0612e60690288a776834811004ce133f326cee
title         - Historiene er nesten for utrolige
Name: 0, dtype: object
Doing seperate train and test data for adressa_no
--
users shape: (455761, 2)
unique (users, articles): (237605, 6793)
userId           cx:ii6jgkx3p8662ju0:26gkbv29my998
id        f9ad33b317728440d5ed1

In [ ]:
import json
with open('datasets/info.json', 'r') as f:
    datasets_info = json.load(f)

In [ ]:
import model.data
import model.plm
importlib.reload(model.data)
importlib.reload(model.plm)
from model.data import get_data_frames, create_graph, create_hetero_graph, create_data_loaders
from model.plm import load_plm_model, get_plm_embeddings, get_items_features

In [ ]:
num_user_features = 32
data_name = 'cit_en'
plm_name = 'enbert'

In [ ]:
with open('datasets/info.json', 'r') as f:
        datasets_info = json.load(f)

users_csv = datasets_info[data_name]['users_csv']
items_csv = datasets_info[data_name]['items_csv']
user_naming = datasets_info[data_name]['user_naming']
item_naming = datasets_info[data_name]['item_naming']
feats_file = datasets_info[data_name]['plm'][plm_name]
epochs = datasets_info[data_name]['epochs']

df_users, df_items = get_data_frames(users_csv, items_csv, user_naming, item_naming)
edges_coo = create_graph(df_users, df_items, user_naming, item_naming)
item_feature_tensor = torch.load(feats_file, map_location=device)
user_features_init = 'zero'
data = create_hetero_graph(edges_coo=edges_coo,
                            user_features_init=user_features_init,
                            user_feature_size=num_user_features,
                            item_feature_tensor=item_feature_tensor)

--
users shape: (26206, 4)
unique (users, articles): (1644, 2154)
contentId                      -3499919498720038879
personId                       -8845298781299428018
url          http://techcrunch.com/2016/06/07/hiri/
lang                                             en
Name: 0, dtype: object
--
items shape: (2154, 4)
unique articles: 2154
contentId                                 -6451309518266745024
title        Ethereum, a Virtual Currency, Enables Transact...
url          http://www.nytimes.com/2016/03/28/business/dea...
lang                                                        en
Name: 0, dtype: object
HeteroData(
  user={
    node_id=[1644],
    x=[1644, 32]
  },
  item={
    node_id=[2154],
    x=[2154, 768]
  },
  (user, rates, item)={ edge_index=[2, 26206] },
  (item, rev_rates, user)={ edge_index=[2, 26206] }
)
user num_nodes: 1644
user num_features: 32
item num_nodes: 2154
item num_features: 768
num_edges user->item: 26206
num_edges item->user: 26206


In [ ]:
feature_tensor = torch.load(feats_file, map_location=device)
feature_tensor.shape

torch.Size([2154, 768])

In [ ]:
# plm_model_name = 'ptbert'
# plm_model, plm_tokenizer = load_plm_model(plm_model_name, device)

In [ ]:
# title_naming = 'title'
# feat_tensor = get_plm_embeddings(plm_model, plm_tokenizer, df_items, title_naming, device)
# print('Output feat_tensor shape:', feat_tensor.shape)

# 4. Data Loaders

In [ ]:
neg_sampling_ratio = 1.0        # hyper: 1, 2
batch_size = 128
num_neighbors = [10, 5]         # hyper: [10, 5], [10, 10], [20, 10], [20, 20]

#### Reload

In [ ]:
# For this, we first split the set of edges into
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, we use 70% of edges for message passing,
# and 30% of edges for supervision.
# We further want to generate fixed negative edges for evaluation with a ratio of 2:1.
# Negative edges during training will be generated on-the-fly.
# We can leverage the `RandomLinkSplit()` transform for this from PyG:
transform = T.RandomLinkSplit(
        num_val=0.0,
        num_test=0.9,
        disjoint_train_ratio=0.3,
        is_undirected=True,                             # new
        neg_sampling_ratio=neg_sampling_ratio,          # negative sampling for val and test
        add_negative_train_samples=False,               # negative samples for train generated on-the-fly
        edge_types=("user", "rates", "item"),
        rev_edge_types=("item", "rev_rates", "user")
    )

train_data, val_data, test_data = transform(data)
print("Training data:")
print("==============")
print(train_data)
print()
print("Validation data:")
print("================")
print(val_data)
print()
print("Test data:")
print("================")
print(test_data)

Training data:
HeteroData(
  user={
    node_id=[1644],
    x=[1644, 32]
  },
  item={
    node_id=[2154],
    x=[2154, 768]
  },
  (user, rates, item)={
    edge_index=[2, 1835],
    edge_label=[786],
    edge_label_index=[2, 786]
  },
  (item, rev_rates, user)={ edge_index=[2, 1835] }
)

Validation data:
HeteroData(
  user={
    node_id=[1644],
    x=[1644, 32]
  },
  item={
    node_id=[2154],
    x=[2154, 768]
  },
  (user, rates, item)={
    edge_index=[2, 2621],
    edge_label=[0],
    edge_label_index=[2, 0]
  },
  (item, rev_rates, user)={ edge_index=[2, 2621] }
)

Test data:
HeteroData(
  user={
    node_id=[1644],
    x=[1644, 32]
  },
  item={
    node_id=[2154],
    x=[2154, 768]
  },
  (user, rates, item)={
    edge_index=[2, 2621],
    edge_label=[47170],
    edge_label_index=[2, 47170]
  },
  (item, rev_rates, user)={ edge_index=[2, 2621] }
)


In [ ]:
print(test_data)
test_data['user', 'rates', 'item'].edge_label.unique()

HeteroData(
  user={
    node_id=[1644],
    x=[1644, 32]
  },
  item={
    node_id=[2154],
    x=[2154, 768]
  },
  (user, rates, item)={
    edge_index=[2, 23586],
    edge_label=[5240],
    edge_label_index=[2, 5240]
  },
  (item, rev_rates, user)={ edge_index=[2, 23586] }
)


tensor([0., 1.])

In [ ]:
print(val_data)
val_data['user', 'rates', 'item'].edge_label.unique()

HeteroData(
  user={
    node_id=[1644],
    x=[1644, 32]
  },
  item={
    node_id=[2154],
    x=[2154, 768]
  },
  (user, rates, item)={
    edge_index=[2, 23586],
    edge_label=[5240],
    edge_label_index=[2, 5240]
  },
  (item, rev_rates, user)={ edge_index=[2, 23586] }
)


tensor([0., 1.])

In [ ]:
print('Train data:')
print('=============')
print('present num_edges user->item:', train_data["user", "rates", "item"].num_edges)
print('to be predicted positive num_edges user->item:', train_data["user", "rates", "item"].edge_label_index.shape[1])
print('to be predicted edge classes:', torch.unique(train_data["user", "rates", "item"].edge_label))
print('negative edge classes [0.] would be generated during training on-the-fly')

Train data:
present num_edges user->item: 16511
to be predicted positive num_edges user->item: 7075
to be predicted edge classes: tensor([1.])
negative edge classes [0.] would be generated during training on-the-fly


In [ ]:
# train data doesn't have negative edges yet
# valid data already has negative edges

In [ ]:
# edge_index        - edges from src nodes (edge_index[0][i]) 
#                   - to dst nodes (edge_index[1][i])
# edge_label        - contains labels of edges, 1 - positive, 0 - negative
# edge_label_index  - same as edge_index, but contains only those edge indices
#                   - on which we have labels, e.g. node234 -> node432: label=1

In [ ]:
# Training data:
# edge_index=[2, 1040140],      - number of edges to construct graph, 70% of edges for message passing,
#                               - edges already present in the graph
# edge_label=[445773],          - number of edges for training, 30% of edges for supervision.
#                               - labels of missing edges, [0, 1] - 0 for negative edge, 1 for positive
# edge_label_index=[2, 445773]  - edges that are absent in the graph for training 

So, we see that all the nodes are present both in training and validation data!

As well as all the item_node features.

Only the edge_index changes: which users connected to which items. Some of them are removed for train and val. 

We are now ready to create a mini-batch loader that will generate subgraphs that can be used as input into our GNN. While this step is not strictly necessary for small-scale graphs, it is absolutely necessary to apply GNNs on larger graphs that do not fit onto GPU memory otherwise. Here, we make use of the loader.LinkNeighborLoader which samples multiple hops from both ends of a link and creates a subgraph from it. Here, edge_label_index serves as the "seed links" to start sampling from.

In [ ]:
print(val_data["user", "rates", "item"].edge_label.unique())

tensor([0., 1.])


In [ ]:
# In the first hop, we sample at most 10 neighbors.
# In the second hop, we sample at most 5 neighbors.
# In addition, during training, we want to sample negative edges on-the-fly with
# a ratio of 2:1.
# We can make use of the `loader.LinkNeighborLoader` from PyG:
# Define seed edges:
edge_label_index = train_data["user", "rates", "item"].edge_label_index
edge_label = train_data["user", "rates", "item"].edge_label

train_loader = LinkNeighborLoader(
        data=train_data,
        num_neighbors=num_neighbors,
        neg_sampling_ratio=neg_sampling_ratio,          # generate train neg samples on the fly
        subgraph_type="bidirectional",
        edge_label_index=(("user", "rates", "item"), train_data["user", "rates", "item"].edge_label_index),
        edge_label=train_data["user", "rates", "item"].edge_label,
        batch_size=batch_size,
        shuffle=True                                 # shuffle=True
    )

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled Train mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "rates", "item"].edge_label_index.size(1) == (neg_sampling_ratio + 1) * batch_size # 2 edges (1 pos + 1 neg) * batch_size
assert len(sampled_data["user", "rates", "item"].edge_label.unique()) == 2
assert sampled_data["user", "rates", "item"].edge_label.min() == 0
assert sampled_data["user", "rates", "item"].edge_label.max() == 1

Sampled Train mini-batch:
HeteroData(
  user={
    node_id=[958],
    x=[958, 32],
    n_id=[958]
  },
  item={
    node_id=[1296],
    x=[1296, 768],
    n_id=[1296]
  },
  (user, rates, item)={
    edge_index=[2, 6475],
    edge_label=[256],
    edge_label_index=[2, 256],
    input_id=[128]
  },
  (item, rev_rates, user)={ edge_index=[2, 6475] }
)


In [ ]:
sampled_data['user', 'rates', 'item'].edge_label.unique()

tensor([0., 1.])

In [ ]:
any(sampled_data['user', 'rates', 'item'].edge_index[0] != sampled_data['item', 'rev_rates', 'user'].edge_index[1])

False

(user, rates, item).edge_index != (item, rev_rates, user).edge_index

Here, we make use of the loader.LinkNeighborLoader which samples multiple hops from both ends of a link and creates a subgraph from it.

In [ ]:
# Define the validation seed edges:
edge_label_index = val_data["user", "rates", "item"].edge_label_index
edge_label = val_data["user", "rates", "item"].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=num_neighbors,
    neg_sampling_ratio=0.0,                                 # already generated for val
    subgraph_type="bidirectional",
    edge_label_index=(("user", "rates", "item"), val_data["user", "rates", "item"].edge_label_index),
    edge_label=val_data["user", "rates", "item"].edge_label,
    batch_size=int((1 + neg_sampling_ratio) * batch_size),  # to account for already sampled neg edges
    shuffle=False                                           # (1 pos + n neg) * batch_size
)

sampled_data = next(iter(val_loader))

print("Sampled Validation mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data["user", "rates", "item"].edge_label_index.size(1) == (neg_sampling_ratio + 1) * batch_size
assert sampled_data["user", "rates", "item"].edge_label.min() >= 0
assert sampled_data["user", "rates", "item"].edge_label.max() <= 1

Sampled Validation mini-batch:
HeteroData(
  user={
    node_id=[964],
    x=[964, 32],
    n_id=[964]
  },
  item={
    node_id=[1295],
    x=[1295, 768],
    n_id=[1295]
  },
  (user, rates, item)={
    edge_index=[2, 7894],
    edge_label=[256],
    edge_label_index=[2, 256],
    input_id=[256]
  },
  (item, rev_rates, user)={ edge_index=[2, 7894] }
)


In [ ]:
for batch_data in tqdm.tqdm(val_loader):
    if len(batch_data['user', 'rates', 'item'].edge_label.unique()) == 2:
        elabel = batch_data['user', 'rates', 'item'].edge_label
        eindex = batch_data['user', 'rates', 'item'].edge_label_index
        print(batch_data)
        print(elabel)
        
        newei = torch.stack([eindex[0][elabel==1], eindex[1][elabel==1]], dim=0)
        print(newei.shape)
        break

 48%|████▊     | 10/21 [00:00<00:00, 122.01it/s]

HeteroData(
  user={
    node_id=[1025],
    x=[1025, 32],
    n_id=[1025]
  },
  item={
    node_id=[1287],
    x=[1287, 768],
    n_id=[1287]
  },
  (user, rates, item)={
    edge_index=[2, 7065],
    edge_label=[256],
    edge_label_index=[2, 256],
    input_id=[256]
  },
  (item, rev_rates, user)={ edge_index=[2, 7065] }
)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.,

In [ ]:
# get indices where it is 1


In [ ]:
sampled_data["user", "rates", "item"].edge_label.unique()

tensor([1.])

In [ ]:
# Define the test seed edges:
edge_label_index = test_data["user", "rates", "item"].edge_label_index
edge_label = test_data["user", "rates", "item"].edge_label

test_loader = LinkNeighborLoader(
    data=test_data,
    num_neighbors=num_neighbors,
    edge_label_index=(("user", "rates", "item"), edge_label_index),
    edge_label=edge_label,
    batch_size=int((neg_sampling_ratio + 1) * batch_size),
    shuffle=False,
)

# sampled_data = next(iter(test_loader))

# print("Sampled Test mini-batch:")
# print("===================")
# print(sampled_data)

# assert sampled_data["user", "rates", "item"].edge_label_index.size(1) == (neg_sampling_ratio + 1) * batch_size
# assert sampled_data["user", "rates", "item"].edge_label.min() >= 0
# assert sampled_data["user", "rates", "item"].edge_label.max() <= 1